# Data Cleaning

The purpose of this code document is to take the raw data from two subreddits (bodyweight fitness and powerlifting) and prepare it for the EDA and modeling process. Cleaning steps include handling null values, removing useless words, combining two text columns into on one column, and applying Natural Language Processing Toolkit (NLTK). The result will be a document-term matrix ready for the EDA process.

In [1]:
#Imported Libraries
import re
import string
import numpy as np
import pandas as pd 

import nltk

from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# To expand Jupyter Notebook's output to more rows and columns
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

In [3]:
# Move CSV files to dataframes
body = pd.read_csv('../datasets/bodyweight_fitness', low_memory=False)
power = pd.read_csv('../datasets/powerlifting', low_memory=False)
# Note: passed low_memory parameter per error message

### Combining the Two Subreddits into One Dataframe and Limiting the Data to Three Columns

The scope of this project is to analyze the text in the subreddits. The only useful columns include the following:
- title
- selftext
- subreddit

In [4]:
dataframe_columns = ['title', 'selftext', 'subreddit']
body = body[dataframe_columns]
power = power[dataframe_columns]

df = pd.concat([body, power]).reset_index().drop(columns='index')
df.head()

,title,selftext,subreddit
0,Vertical Pushing in Which Direction?,Currently I am only doing horizontal pushing i...,bodyweightfitness
1,Pelican Curls and Shoulder mobility,So as the title says i think the pelican curl ...,bodyweightfitness
2,Use fat mobilizer to shed your extra kilos,[removed],bodyweightfitness
3,Can't bring get legs low enough for planche,"Hey, guys, I'm 14 and I started working out wi...",bodyweightfitness
4,Just started the RR 6 Days ago and I went from...,I know it's not super exciting or that great. ...,bodyweightfitness


### Cleaning Null Values

According to the code below, the selftext column contains 2,796 null values while the title column has 0 null values. In order to retain the rows with text in the 'title' but not in the 'selftext', I replace the null values with an empty string. In a subsequent step I will combine the two columns so all the text is in one column.

In [5]:
print(df.isnull().sum())

title           0
selftext     2796
subreddit       0
dtype: int64


In [6]:
# Fill NA with an empty string
df.selftext.fillna('', inplace=True)

In [7]:
# Confirmed that the null values were replaced with an empty string
print(df.isnull().sum())

title        0
selftext     0
subreddit    0
dtype: int64


After evaluating the value counts, I noticed that "selftext" has two values that are equivalent to null values. These values were also replaced with en empty string. The values and number of instances were the following:
- deleted: 100
- removed: 6,049

In [8]:
removing_values = ['[removed]', 
                   '[deleted]',
                   ]


df.replace(to_replace = removing_values,  
            value ="",
            inplace=True)

The evaluation into the value counts also revealed a handful of repeated posts. Subreddit writers will often publish the same thread on a periodic bases (weekly, daily, etc.). My concern is that the repeated posts will influence the model in a way that reduces its predicability; however, I decided to leave the posts in the data to more accurately reflect the Reddit environment. See below for the top three most frequent examples. 

In [9]:
print(df.selftext.value_counts()[1:4])

**A sorta kinda daily open thread to use as an alternative to posting on the main board.You should post here for:**\n\n*  PRs \n\n*  Formchecks\n\n*  Rudimentary discussion or questions\n\n*  General conversation with other users\n\n*  Memes, funnies, and general bollocks not appropriate to the main board\n\n*  If you have suggestions for the subreddit, let us know!\n\n*  **This thread now defaults to "new" sorting.**                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

### Creating an All Text Column

In [10]:
df['all_text'] = df.title + ' ' + df.selftext

### Changing Target Variable to 0/1

In [11]:
# bodyweight fitness = 0
# powerlifting = 1
df.subreddit = df.subreddit.map({'bodyweightfitness':0,
                                     'powerlifting':1})

### Natural Language Processing

To prepare the data for the model the equations below will accomplish the following:
- Change all words to lowercase
- Remove numbers
- Remove the reddit names
- Remove website url
- Remove English stop words

In [12]:
nltk.download('stopwords')
corpus = []
words = set(nltk.corpus.words.words())

for i in range(0, 20_000):
    text = re.sub('[^a-zA-Z]', ' ', df['all_text'][i])
    text = text.lower()
    pattern = r"http\S+"
    text = re.sub(pattern, "", text)
    text = re.sub('(bodyweight)|(powerlifting)', "", text)
    text = text.split()

    ps = PorterStemmer()
    
    text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    
    if i % 2000 == 0:
            print(f'Post {i} of 20000')
    
    corpus.append(text)
    
# Code Reference:

# remove words -- https://www.kite.com/python/answers/how-to-use-re.sub()-in-python
# remove punctuation --> https://stackoverflow.com/questions/18429143/strip-punctuation-with-regex-python/50985687
# stop words and list comprehension --> https://towardsdatascience.com/build-and-compare-3-models-nlp-sentiment-prediction-67320979de61
# remove youtube url websites - https://www.youtube.com/watch?v=O2onA4r5UaY
# porter stemmer guidance --> https://pythonprogramming.net/stemming-nltk-tutorial/
# list comprehension --> https://www.learnpython.dev/03-intermediate-python/20-advanced-looping/30-other-comprehensions/
# join python list --> https://www.w3schools.com/python/ref_string_join.asp

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brianmendes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Post 0 of 20000
Post 2000 of 20000
Post 4000 of 20000
Post 6000 of 20000
Post 8000 of 20000
Post 10000 of 20000
Post 12000 of 20000
Post 14000 of 20000
Post 16000 of 20000
Post 18000 of 20000


In [13]:
# Save the scrubbed list the 'all_text' column
text = pd.DataFrame(corpus)
df['all_text'] = text

### Vectorize the Data

I vectorized the data in two way: Tf-idf Vectorizer and CountVectorizer. Both methods have a min_df=3 and use stop words. 

In [14]:
tvec = TfidfVectorizer(min_df=3,stop_words='english')

tvec_title = tvec.fit(df['all_text'])

data_df_tvec = pd.DataFrame(tvec.transform(df['all_text']).todense(), 
                       columns=tvec.get_feature_names())
data_df_tvec['subreddit'] = df['subreddit']
data_df_tvec.head()

,aa,aaron,aautomoder,ab,aba,abandon,abb,abd,abdomen,abdomin,abdomini,abduct,abil,abit,abl,abpu,abroad,absent,absolut,absorb,absurd,abt,abus,abysm,ac,academ,academi,acc,acceler,accept,accesori,access,accessori,accid,accident,acclim,accommod,accomod,accompani,accomplish,accord,accordingli,account,accredit,accumul,accur,accuraci,accus,accustom,ace,acessori,ach,achi,achiev,achil,achiv,acid,acknowledg,acl,acn,acquaint,acquir,acquisit,act,action,activ,actor,actual,acut,ad,adam,adapt,add,addict,addit,address,adduct,adductor,adept,adequ,adhd,adher,adida,adipow,adjust,admin,admir,admit,admittedli,ado,adoni,adopt,ador,adrenalin,adrien,adult,adv,advanc,advantag,adventur,advers,advertis,advic,advis,advisor,advoc,ae,aerob,aesthet,af,afaik,affac,affair,affect,affili,afford,aforement,afraid,aftermath,afternoon,afterthought,afterward,age,agent,aggrav,aggress,agil,agit,ago,agon,agoni,agonis,agre,ah,ahead,ai,aid,aim,air,airbnb,ak,aka,akin,akward,al,ala,alan,alarm,albeit,album,alcohol,alert,alex,alexand,alexey,algorithm,alhazov,alibaba,align,aliv,alleg,allen,allergi,allevi,allianc,allot,allow,almond,alon,alongsid,alot,alphadestini,alreadi,alright,alsruh,alt,alter,altern,altogeth,altough,alway,ama,amanda,amateur,amaz,amazingli,amazon,ambassador,ambiti,amd,amend,america,american,ami,amino,aml,ammonia,amounth,amout,amp,ampl,amplifi,amrap,ana,anabol,anaerob,analys,analysi,analyz,anatom,anatomi,anchor,ancient,andi,andrea,andrew,andrey,android,anecdot,angel,angelo,anger,angl,angri,anim,ankl,anniversari,announc,annoy,annual,anonym,anorexia,anoth,answer,antagonist,anterior,anthoni,anti,anticip,antidepress,antonio,antranik,anxieti,anxiou,anybodi,anymor,anyon,anyth,anytim,anywher,anywho,ap,apa,apart,apf,api,apnea,apolog,apologis,app,appar,apparel,appart,appeal,appear,appendectomi,appetit,appl,appleton,appli,applic,appoint,appreci,approach,appropri,approv,approx,approxim,apreci,april,aprox,apt,apu,ar,arbitrari,arc,arch,archer,archiv,arduou,area,arent,argentina,argu,arguabl,argument,arian,aris,arm,armi,armor,armpit,armstrong,armwrestl,arnold,arrang,array,arremoni,arriv,ars,arsecheekosi,art,arthriti,articl,artifici,artist,asa,asap,ascend,ascent,asham,ashtanga,ashton,asia,asian,asic,asid,asin,ask,asleep,asolfir,aspect,aspir,ass,assault,assembl,assess,asset,asshol,assign,assist,associ,assort,assum,assumpt,assur,asterisk,asthma,astonish,aswel,asymmetr,asymmetri,ate,atg,athelet,athlean,athleanx,athlet,athletic,atleast,atm,atmospher,atrophi,attach,attack,attain,attempt,attend,attent,attitud,attract,attribut,atwood,atwr,au,aud,audienc,audio,aug,august,aussi,austin,australia,australian,austria,author,autism,auto,automat,automod,autoplay,autoregul,autumn,auxiliari,avail,avatar,averag,avid,avocado,avoid,aw,awak,awar,award,away,awe,awesom,awhil,awkward,awkwardli,axl,ay,ayurved,ayurveda,az,ba,babi,backdown,backflip,background,backlev,backoff,backpack,backsid,backstag,backstori,backup,backward,backyard,bacon,bacteria,bad,badass,badli,badminton,baffl,bag,bagel,baggi,bail,bake,bakkelund,balanc,bald,ball,balloon,ballsi,ban,banana,band,bane,bang,bank,bar,barbel,bare,barebon,barefoot,barley,barrier,basal,base,basebal,basebar,baseblock,baselin,basement,bash,basi,basic,basketbal,bass,bastard,bat,bath,bathroom,batman,batteri,battl,bavoil,bay,bb,bbb,bbm,bbr,bc,bcaa,bcg,bd,bday,beach,beam,bean,bear,bearabl,beast,beat,beaten,beauti,...,unlimit,unload,unlock,unmotiv,unnatur,unnecessari,unoffici,unpleas,unpopular,unrack,unreal,unrealist,unrel,unsaf,unsanct,unsatisfi,unstabl,unstopp,unstructur,unsur,untest,untrain,unus,unusu,unwant,unweight,uo,upa,upcom,upcycl,updat,upgrad,uphil,upload,upper,upperbodi,upright,upset,upsid,upvot,upward,ur,urg,urgent,urin,url,usa,usabl,usag,usapl,usd,use,useless,user,usernam,usp,uspa,uspc,usual,utah,utf,util,utilis,utm,uu,uy,va,vacat,vacuum,vadnal,vagu,vain,valentin,valgu,valid,valley,valsalva,valu,valuabl,van,vanilla,vaniti,vari,variabl,varianc,variant,variat,varieti,variou,vascular,vast,vastli,vastu,vb,vc,ved,veg,vega,vegan,veget,vegetarian,veggi,vehi

In [16]:
cvec = CountVectorizer(min_df=3, stop_words='english')

cvec_title = cvec.fit_transform(df['all_text'])
data_df = pd.DataFrame(cvec_title.toarray(), columns=cvec.get_feature_names())
data_df['subreddit'] = df['subreddit']
data_df.head()

,aa,aaron,aautomoder,ab,aba,abandon,abb,abd,abdomen,abdomin,abdomini,abduct,abil,abit,abl,abpu,abroad,absent,absolut,absorb,absurd,abt,abus,abysm,ac,academ,academi,acc,acceler,accept,accesori,access,accessori,accid,accident,acclim,accommod,accomod,accompani,accomplish,accord,accordingli,account,accredit,accumul,accur,accuraci,accus,accustom,ace,acessori,ach,achi,achiev,achil,achiv,acid,acknowledg,acl,acn,acquaint,acquir,acquisit,act,action,activ,actor,actual,acut,ad,adam,adapt,add,addict,addit,address,adduct,adductor,adept,adequ,adhd,adher,adida,adipow,adjust,admin,admir,admit,admittedli,ado,adoni,adopt,ador,adrenalin,adrien,adult,adv,advanc,advantag,adventur,advers,advertis,advic,advis,advisor,advoc,ae,aerob,aesthet,af,afaik,affac,affair,affect,affili,afford,aforement,afraid,aftermath,afternoon,afterthought,afterward,age,agent,aggrav,aggress,agil,agit,ago,agon,agoni,agonis,agre,ah,ahead,ai,aid,aim,air,airbnb,ak,aka,akin,akward,al,ala,alan,alarm,albeit,album,alcohol,alert,alex,alexand,alexey,algorithm,alhazov,alibaba,align,aliv,alleg,allen,allergi,allevi,allianc,allot,allow,almond,alon,alongsid,alot,alphadestini,alreadi,alright,alsruh,alt,alter,altern,altogeth,altough,alway,ama,amanda,amateur,amaz,amazingli,amazon,ambassador,ambiti,amd,amend,america,american,ami,amino,aml,ammonia,amounth,amout,amp,ampl,amplifi,amrap,ana,anabol,anaerob,analys,analysi,analyz,anatom,anatomi,anchor,ancient,andi,andrea,andrew,andrey,android,anecdot,angel,angelo,anger,angl,angri,anim,ankl,anniversari,announc,annoy,annual,anonym,anorexia,anoth,answer,antagonist,anterior,anthoni,anti,anticip,antidepress,antonio,antranik,anxieti,anxiou,anybodi,anymor,anyon,anyth,anytim,anywher,anywho,ap,apa,apart,apf,api,apnea,apolog,apologis,app,appar,apparel,appart,appeal,appear,appendectomi,appetit,appl,appleton,appli,applic,appoint,appreci,approach,appropri,approv,approx,approxim,apreci,april,aprox,apt,apu,ar,arbitrari,arc,arch,archer,archiv,arduou,area,arent,argentina,argu,arguabl,argument,arian,aris,arm,armi,armor,armpit,armstrong,armwrestl,arnold,arrang,array,arremoni,arriv,ars,arsecheekosi,art,arthriti,articl,artifici,artist,asa,asap,ascend,ascent,asham,ashtanga,ashton,asia,asian,asic,asid,asin,ask,asleep,asolfir,aspect,aspir,ass,assault,assembl,assess,asset,asshol,assign,assist,associ,assort,assum,assumpt,assur,asterisk,asthma,astonish,aswel,asymmetr,asymmetri,ate,atg,athelet,athlean,athleanx,athlet,athletic,atleast,atm,atmospher,atrophi,attach,attack,attain,attempt,attend,attent,attitud,attract,attribut,atwood,atwr,au,aud,audienc,audio,aug,august,aussi,austin,australia,australian,austria,author,autism,auto,automat,automod,autoplay,autoregul,autumn,auxiliari,avail,avatar,averag,avid,avocado,avoid,aw,awak,awar,award,away,awe,awesom,awhil,awkward,awkwardli,axl,ay,ayurved,ayurveda,az,ba,babi,backdown,backflip,background,backlev,backoff,backpack,backsid,backstag,backstori,backup,backward,backyard,bacon,bacteria,bad,badass,badli,badminton,baffl,bag,bagel,baggi,bail,bake,bakkelund,balanc,bald,ball,balloon,ballsi,ban,banana,band,bane,bang,bank,bar,barbel,bare,barebon,barefoot,barley,barrier,basal,base,basebal,basebar,baseblock,baselin,basement,bash,basi,basic,basketbal,bass,bastard,bat,bath,bathroom,batman,batteri,battl,bavoil,bay,bb,bbb,bbm,bbr,bc,bcaa,bcg,bd,bday,beach,beam,bean,bear,bearabl,beast,beat,beaten,beauti,...,unlimit,unload,unlock,unmotiv,unnatur,unnecessari,unoffici,unpleas,unpopular,unrack,unreal,unrealist,unrel,unsaf,unsanct,unsatisfi,unstabl,unstopp,unstructur,unsur,untest,untrain,unus,unusu,unwant,unweight,uo,upa,upcom,upcycl,updat,upgrad,uphil,upload,upper,upperbodi,upright,upset,upsid,upvot,upward,ur,urg,urgent,urin,url,usa,usabl,usag,usapl,usd,use,useless,user,usernam,usp,uspa,uspc,usual,utah,utf,util,utilis,utm,uu,uy,va,vacat,vacuum,vadnal,vagu,vain,valentin,valgu,valid,valley,valsalva,valu,valuabl,van,vanilla,vaniti,vari,variabl,varianc,variant,variat,varieti,variou,vascular,vast,vastli,vastu,vb,vc,ved,veg,vega,vegan,veget,vegetarian,veggi,vehi

### Save DataFrames as CSV Files

In [17]:
# data_df_tvec.to_csv('../datasets/model_ready_data_tvec.csv', index=False)
data_df.to_csv('../datasets/model_ready_data.csv', index=False) 
# df.to_csv('../datasets/cleaned_data.csv', index=False)
 